In [1]:
from pathlib import Path
import pandas as pd

## столбцы 
- product - Наименование товара
- group_of_products - Номенклатурная группа
- city - Город
- receipt - Номер чека ККМ
- date - Дата продажи
- store - АЗС
- sold_quantity - Количество проданных товаров
- cost_VAT - Закупочная сумма (с НДС)
- Розничная сумма
- Сумма наценки

In [2]:
data_folder = input("Путь к файлам(относительный): ")

Путь к файлам(относительный): folder


### !! Очищаем папку от скрытых файлов, иначе будет ошибка 

In [3]:
# путь
p = Path(data_folder)

# находим xlsx
files = p.glob('*.xlsx')

# df
df_main = pd.concat([pd.read_excel(file, header=None, skiprows=2, skipfooter=1) for file in files])

In [69]:
# Делаем копию объединения файлов - для быстрого возврата к исходнику 
df = df_main.copy()

In [70]:
df.shape

(1640070, 10)

In [71]:
# Удаляем строки с nan столбцах -выручка, себестоимость 
df = df.dropna(subset=[7, 8], thresh=1)

In [72]:
df.shape

(1640037, 10)

- после удаления строк содержащих nan в столбцах -выручка и себестоимость, размер таблицы не изменился 

In [73]:
df.head(4)

,0,1,2,3,4,5,6,7,8,9
0,Чек ККМ ГС00-000001 от 01.01.2021 0:26:25,NaN,NaN,207546.0,01.01.2021 0:26:25,АЗС №2 г.Керчь,2,41.76,100.0,58.24
1,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЛАЯ ВОДА,NaN,NaN,NaN,NaN,1,26.92,40.0,13.08
2,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЛАЯ ВОДА,Керчь г,NaN,NaN,NaN,1,26.92,40.0,13.08
3,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,NaN,NaN,NaN,NaN,1,14.84,60.0,45.16


In [74]:
# Заполняем nan
df[[2, 3, 4, 5]] = df[[2, 3, 4, 5]].fillna(method='ffill')

In [75]:
#  удаляем лишние строки Содержащие чек, возврат, отчёт 
df = df[~df[0].str.contains('Чек|Возврат|Отчет')]
df = df[~df[0].str.contains('Возврат')]
df = df[~df[0].str.contains('Отчет')]

In [11]:
df.head(4) 

,0,1,2,3,4,5,6,7,8,9
1,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЛАЯ ВОДА,NaN,207546.0,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,26.92,40.0,13.08
2,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЛАЯ ВОДА,Керчь г,207546.0,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,26.92,40.0,13.08
3,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,Керчь г,207546.0,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,14.84,60.0,45.16
4,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,Керчь г,207546.0,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,14.84,60.0,45.16


In [76]:
# Именуем столбцы 
df.columns = ['product', 'group_of_products', 'city', 'receipt', 'date', 'store', 'sold_quantity', 
              'cost_VAT', 'Розничная сумма', 'Сумма наценки']

In [77]:

df.loc[(df['cost_VAT'] ==0) & (df['Розничная сумма'] ==0)]

,product,group_of_products,city,receipt,date,store,sold_quantity,cost_VAT,Розничная сумма,Сумма наценки


In [78]:
df.head(4)

,product,group_of_products,city,receipt,date,store,sold_quantity,cost_VAT,Розничная сумма,Сумма наценки
1,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЛАЯ ВОДА,NaN,207546.0,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,26.92,40.0,13.08
2,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЛАЯ ВОДА,Керчь г,207546.0,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,26.92,40.0,13.08
3,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,Керчь г,207546.0,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,14.84,60.0,45.16
4,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,Керчь г,207546.0,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,14.84,60.0,45.16


In [53]:
# Тип данных чека в число 
df = df.astype({"receipt": "Int64"})

In [34]:
df.head(4)

,product,group_of_products,city,receipt,date,store,sold_quantity,cost_VAT,Розничная сумма,Сумма наценки
1,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЛАЯ ВОДА,NaN,207546,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,26.92,40.0,13.08
2,"Вода мин. Крымская слабогаз. 1,0л ПЭТ",БЕЛАЯ ВОДА,Керчь г,207546,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,26.92,40.0,13.08
3,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,Керчь г,207546,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,14.84,60.0,45.16
4,Кофе АМЕРИКАНО 200 мл АЗС 2 АЗС 4,КОФЕЙНЫЕ НАПИТКИ,Керчь г,207546,01.01.2021 0:26:25,АЗС №2 г.Керчь,1,14.84,60.0,45.16


In [54]:
# Создаем новые столбцы 
df['profit'] = df['Сумма наценки'] / df['sold_quantity']
df['Стоимость единицы'] = df['cost_VAT'] / df['sold_quantity']

In [55]:
# Удаляем ненужные столбцы 
df = df.drop(['product', 'group_of_products', 
               'Розничная сумма', 'Сумма наценки', 'cost_VAT', 'sold_quantity'], axis=1)   

In [56]:
# удаляем дубликаты 
df = df.drop_duplicates()

In [38]:
df.head(4)

,city,receipt,date,store,profit,Стоимость единицы
1,NaN,207546,01.01.2021 0:26:25,АЗС №2 г.Керчь,13.08,26.92
2,Керчь г,207546,01.01.2021 0:26:25,АЗС №2 г.Керчь,13.08,26.92
3,Керчь г,207546,01.01.2021 0:26:25,АЗС №2 г.Керчь,45.16,14.84
6,Керчь г,207548,01.01.2021 0:28:27,АЗС №2 г.Керчь,6.91,108.09


In [57]:
city = {'АЗС №2 г.Керчь': "Керчь", 'АЗС № 1 Героев Сталинграда 8А': "Симферополь", 
        'АЗС №11 г.Симферополь, ул. Внешняя, д.28, литера Ж': "Симферополь", 'АЗС №3 г.Севастополь': "Севастополь", 
        'АЗС №4 г.Симферополь': "Симферополь"}
store = {'АЗС №2 г.Керчь': "АЗС №2", 'АЗС № 1 Героев Сталинграда 8А': "АЗС №1", 
        'АЗС №11 г.Симферополь, ул. Внешняя, д.28, литера Ж': "АЗС №11", 'АЗС №3 г.Севастополь': "АЗС №3", 
        'АЗС №4 г.Симферополь': "АЗС №4"}

In [58]:
df['city'] = df['store'].map(city)
df['store'] = df['store'].map(store)

In [41]:
df.head(4)

,city,receipt,date,store,profit,Стоимость единицы
1,Керчь,207546,01.01.2021 0:26:25,АЗС №2,13.08,26.92
2,Керчь,207546,01.01.2021 0:26:25,АЗС №2,13.08,26.92
3,Керчь,207546,01.01.2021 0:26:25,АЗС №2,45.16,14.84
6,Керчь,207548,01.01.2021 0:28:27,АЗС №2,6.91,108.09


In [59]:
# Заменяем nan нулями 
df = df.fillna(0)

In [60]:
# Смотрим nan по всем столбцам 
df.isnull().sum()

city                 0
receipt              0
date                 0
store                0
profit               0
Стоимость единицы    0
dtype: int64

In [61]:
df.shape

(815530, 6)

In [27]:
df.to_excel(f'{data_folder}/merged.xlsx') 